In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="Bar7"
JobNameAbbrev = "B7"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                    #    "Seed":list(range(0,100)),
                       "Seed":list([]),                       
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.035],
                       "Type": ["Classification"],
                       "Partition": ["largemem"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["11:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["150G"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Passive Learning

In [3]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list([0, 2, 3, 12, 25, 31, 37, 44, 56, 75, 79, 85, 88, 91, 95]),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["PassiveLearning"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

/var/folders/10/2lfzfs1j0j933_mjkrskp5kh0000gq/T/ipykernel_44509/1735281614.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline


# Include/exclude Random Forest Simulations

In [4]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list([0, 2, 3, 12, 25, 31, 37, 44, 56, 75, 79, 85, 88, 91, 95]),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [5]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI1_", "_PL", regex=True)
    )

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

---

# Only run these ones

In [6]:
# # ### Simulations that failed ###
# FilterText = ['0BC_DA0.04', '1BC_DA0.04', '2BC_DA0.04', '3BC_DA0.04', '4BC_DA0.04', '5BC_DA0.04', '6BC_DA0.04', '7BC_DA0.04', '8BC_DA0.04', '9BC_DA0.04', '10BC_DA0.04', '11BC_DA0.04', '12BC_DA0.04', '14BC_DA0.04', '17BC_DA0.04', '18BC_DA0.04', '20BC_DA0.04', '22BC_DA0.04', '23BC_DA0.04', '27BC_DA0.04', '28BC_DA0.04', '29BC_DA0.04', '30BC_DA0.04', '32BC_DA0.04', '33BC_DA0.04', '34BC_DA0.04', '36BC_DA0.04', '37BC_DA0.04', '39BC_DA0.04', '40BC_DA0.04', '43BC_DA0.04', '44BC_DA0.04', '45BC_DA0.04', '47BC_DA0.04', '48BC_DA0.04', '49BC_DA0.04', '50BC_DA0.04', '51BC_DA0.04', '52BC_DA0.04', '53BC_DA0.04', '57BC_DA0.04', '58BC_DA0.04', '61BC_DA0.04', '62BC_DA0.04', '63BC_DA0.04', '64BC_DA0.04', '69BC_DA0.04', '70BC_DA0.04', '73BC_DA0.04', '75BC_DA0.04', '82BC_DA0.04', '86BC_DA0.04', '87BC_DA0.04', '88BC_DA0.04', '89BC_DA0.04', '90BC_DA0.04', '91BC_DA0.04', '95BC_DA0.04', '97BC_DA0.04', '98BC_DA0.04', '0BC_UA0.04', '1BC_UA0.04', '2BC_UA0.04', '3BC_UA0.04', '4BC_UA0.04', '5BC_UA0.04', '6BC_UA0.04', '7BC_UA0.04', '8BC_UA0.04', '9BC_UA0.04', '10BC_UA0.04', '12BC_UA0.04', '14BC_UA0.04', '17BC_UA0.04', '18BC_UA0.04', '20BC_UA0.04', '22BC_UA0.04', '23BC_UA0.04', '27BC_UA0.04', '28BC_UA0.04', '29BC_UA0.04', '30BC_UA0.04', '32BC_UA0.04', '33BC_UA0.04', '34BC_UA0.04', '37BC_UA0.04', '39BC_UA0.04', '40BC_UA0.04', '43BC_UA0.04', '44BC_UA0.04', '45BC_UA0.04', '47BC_UA0.04', '48BC_UA0.04', '49BC_UA0.04', '50BC_UA0.04', '51BC_UA0.04', '52BC_UA0.04', '53BC_UA0.04', '57BC_UA0.04', '61BC_UA0.04', '62BC_UA0.04', '63BC_UA0.04', '64BC_UA0.04', '69BC_UA0.04', '70BC_UA0.04', '73BC_UA0.04', '75BC_UA0.04', '77BC_UA0.04', '82BC_UA0.04', '83BC_UA0.04', '86BC_UA0.04', '87BC_UA0.04', '89BC_UA0.04', '90BC_UA0.04', '91BC_UA0.04', '95BC_UA0.04', '97BC_UA0.04', '98BC_UA0.04', '99BC_UA0.04']

# ### Rerun these simulations ###
# ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector

---

In [7]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,Partition,Time,Memory,JobName,Output
0,Bar7,0,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,0B7_PLA0,Bar7/RandomForestClassification/Raw/0B7_PLA0.pkl
1,Bar7,0,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,0B7_RFA0,Bar7/RandomForestClassification/Raw/0B7_RFA0.pkl
2,Bar7,2,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,2B7_PLA0,Bar7/RandomForestClassification/Raw/2B7_PLA0.pkl
3,Bar7,2,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,2B7_RFA0,Bar7/RandomForestClassification/Raw/2B7_RFA0.pkl
4,Bar7,3,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,3B7_PLA0,Bar7/RandomForestClassification/Raw/3B7_PLA0.pkl
5,Bar7,3,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,3B7_RFA0,Bar7/RandomForestClassification/Raw/3B7_RFA0.pkl
6,Bar7,12,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,12B7_PLA0,Bar7/RandomForestClassification/Raw/12B7_PLA0.pkl
7,Bar7,12,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,12B7_RFA0,Bar7/RandomForestClassification/Raw/12B7_RFA0.pkl
8,Bar7,25,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,25B7_PLA0,Bar7/RandomForestClassification/Raw/25B7_PLA0.pkl
9,Bar7,25,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,25B7_RFA0,Bar7/RandomForestClassification/Raw/25B7_RFA0.pkl


# Save Parameter Vector

In [8]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)